# Background system of Si production

First we import the packages we need.

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

<IPython.core.display.Javascript object>

Now we connect to the Platform.

In [2]:
mp = ixmp.Platform()

Creating a scenario which contains our model, which will contain the input data and results.

In [3]:
scenario = message_ix.Scenario(mp, model='Production system', 
                               scenario='baseline', version='new')

Basic characteristics of the model: defining time, space and the structure of the production system.

In [4]:
#defining time:

history = [2020]
model_horizon = [2030, 2040, 2050]
scenario.add_horizon(
    year=history + model_horizon,
    firstmodelyear=model_horizon[0]
)

#our spatial dimension consits on 1 node:

country = 'Norway'
scenario.add_spatial_sets({'country': country})

In [5]:
#finally we define the commodities, levels, technologuies and modes of operation, which define how certain technologies operate.

scenario.add_set("commodity", ["scrap", "MGS", "solar_pv"])
scenario.add_set("level", ["recovery", "storage", "renewable"])
scenario.add_set("technology", ["residues", 'SAF', "alumino", "production"])
scenario.add_set("mode", "standard")

In this model, we want to satisfy the demand of one material (silicon). To establish demand, we need a set of assumptions:

- The approach taken by Eric Willians in "Forecasting material and economic flows in the global production chain for silicon" is followed.

$\text{Production}_{t}=\text{Production}_{0}\cdot(1+g)^t$

(Assumes future demand can be estimated with relatively hugh degree of confidence by looking at longer trends, and applies a growth of 3,5% yearly)
    

In [6]:
demand_2030=320000*(1+0.035)**10.0
demand_2040=320000*(1+0.035)**20.0
demand_2050=320000*(1+0.035)**30.0

In [7]:
production_0=320000
#tonnes of Si produced in NO, estimated a 4% of the global figure, sources: https://www.statista.com/statistics/573585/global-silicon-production/ and https://pubs.usgs.gov/periodicals/mcs2021/mcs2021-silicon.pdf
g=3.5
silicon_demand = pd.DataFrame({
        'node': country,
        'commodity': 'solar_pv',
        'level': 'renewable',
        'year': model_horizon,
        'time': 'year',
        'value': [demand_2030,demand_2040,demand_2050],
        'unit': 't',
    })

In [8]:
silicon_demand

node commodity      level  year  time          value unit
0  Norway  solar_pv  renewable  2030  year  451391.603399    t
1  Norway  solar_pv  renewable  2040  year  636732.436309    t
2  Norway  solar_pv  renewable  2050  year  898173.985505    t

In [9]:
# We use add_par for adding data to a MESSAGEix parameter.
scenario.add_par("demand", silicon_demand)

In [10]:
year_df = scenario.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']

base = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': '-',
}

base_input = make_df(base, node_origin=country, time_origin='year')
base_output = make_df(base, node_dest=country, time_dest='year')

In [11]:
# Now we define efficiencies for the technologies (efficiency=output/input): "residues", 'SAF', "alumino", 'production'.

In [12]:
# Assume efficiencies are 80% for extraction of raw material in residues.
# Assume 90% efficiency in the SAF process.

residues_out = make_df(base_output, technology='residues', commodity='scrap', 
                   level='recovery', mode="standard", value=0.8)
scenario.add_par('output', residues_out)

SAF_out = make_df(base_output, technology='SAF', commodity='MGS', 
                   level='storage', value=0.9)
scenario.add_par('output', SAF_out)

alumino_out = make_df(base_output, technology='alumino', commodity='MGS', 
                   level='storage', value=1.0)
scenario.add_par('output', alumino_out)

alumino_in = make_df(base_input, technology='alumino', commodity='scrap',  
                  level='recovery', value=1.0)
scenario.add_par('input', alumino_in)

production_out = make_df(base_output, technology='production', commodity='solar_pv', 
                   level='renewable', value=1.0)
scenario.add_par('output', production_out)

production_in = make_df(base_input, technology='production', commodity='MGS',  
                  level='storage', value=1.0)
scenario.add_par('input', production_in)


In [13]:
scenario.add_par("interestrate", model_horizon, value=0.05, unit='-')

base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
}

# We assume fixed costs and investment costs are equal, because the production of both routes is developed in the same plant, and therefore we exclude these costs.
# The cost of acquiring residues is disregarded.
# The cost of alumino is assumed to be 50% higher. This is because the technology has only been implemented on a Pilot scale, and there is huge uncertainty associated to the costs related to sourcing and purification of raw materials.

costs = {
    'alumino': 3000,
    'SAF':  2000,
}

for tec, val in costs.items():
    df = make_df(base_var_cost, technology=tec, value=val)
    scenario.add_par('var_cost', df)

In [14]:
# Defining other constraints

# first we introduce the emission of CO2 and the emission category GHG
scenario.add_set('emission', 'CO2')
scenario.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions
scenario_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'tCO2/ton',
}

# adding new units to the model library (needed only once)
mp.add_unit('tCO2/ton')



emissions = {
    'SAF': ('CO2', 6.3), # units: tCO2/ton
    'alumino':  ('CO2', 5.1), # units: tCO2/ton
}

for tec, (species, val) in emissions.items():
    emission_factor = make_df(scenario_emission_factor, technology=tec, emission=species, value=val) 
    scenario.add_par('emission_factor', emission_factor)
    

# Emissions from master's thesis Table D.1
# https://ntnuopen.ntnu.no/ntnu-xmlui/handle/11250/2824786

In [15]:
scenario.par("emission_factor")

node_loc technology  year_vtg  year_act      mode emission  value      unit
0    Norway        SAF      2020      2030  standard      CO2    6.3  tCO2/ton
1    Norway        SAF      2020      2040  standard      CO2    6.3  tCO2/ton
2    Norway        SAF      2020      2050  standard      CO2    6.3  tCO2/ton
3    Norway        SAF      2030      2030  standard      CO2    6.3  tCO2/ton
4    Norway        SAF      2030      2040  standard      CO2    6.3  tCO2/ton
5    Norway        SAF      2030      2050  standard      CO2    6.3  tCO2/ton
6    Norway        SAF      2040      2040  standard      CO2    6.3  tCO2/ton
7    Norway        SAF      2040      2050  standard      CO2    6.3  tCO2/ton
8    Norway        SAF      2050      2050  standard      CO2    6.3  tCO2/ton
9    Norway    alumino      2020      2030  standard      CO2    5.1  tCO2/ton
10   Norway    alumino      2020      2040  standard      CO2    5.1  tCO2/ton
11   Norway    alumino      2020      2050  standard      CO2    5.1  tCO2/ton
12   Norway    alumino      2030      2030  standard      CO2    5.1  tCO2/ton
13   Norway    alumino      2030      2040  standard      CO2    5.1  tCO2/ton
14   Norway    alumino      2030      2050  standard      CO2    5.1  tCO2/ton
15   Norway    alumino      2040      2040  standard      CO2    5.1  tCO2/ton
16   Norway    alumino      2040      2050  standard      CO2    5.1  tCO2/ton
17   Norway    alumino      2050      2050  standard      CO2    5.1  tCO2/ton

In [16]:
# Defining technical lifetime

base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}

lifetime = {
    'alumino': 40,
    'SAF': 40,
}

for tec, val in lifetime.items():
    df = make_df(base_technical_lifetime, technology=tec, value=val)
    scenario.add_par('technical_lifetime', df)

In [17]:
# Defining a bound on emissions (from the Norwegian emissions inventory)

scenario.add_cat('year', 'cumulative', history + model_horizon)
scenario.add_par('bound_emission', [country, 'GHG', 'all', 'cumulative'],
             value=2554900., unit='tCO2')

# Introducing a carbon tax

base_tax_emission = {
    'node': 'Norway',
    'type_year': model_horizon,
    'type_tec': 'all',
    'unit': 'USD/tCO2',
    'type_emission': 'GHG',
    'value': 50
}

# in case you need to add the new unit to the platform
mp.add_unit('USD/tCO2')

scenario.add_cat('year', 2030, 2030)
scenario.add_cat('year', 2040, 2040)
scenario.add_cat('year', 2050, 2050)

tax_emission = make_df(base_tax_emission)
scenario.add_par('tax_emission', tax_emission)
scenario.par('tax_emission')



node type_emission type_tec type_year  value      unit
0  Norway           GHG      all      2030   50.0  USD/tCO2
1  Norway           GHG      all      2040   50.0  USD/tCO2
2  Norway           GHG      all      2050   50.0  USD/tCO2

In [18]:
# Defining limited residues

commodity= 'scrap'
level = 'recovery'
scenario.add_set('commodity', commodity)
scenario.add_set('level', level)
scenario.add_set('level_resource', level)
scenario.add_set('grade', ['dross', 'post-consumer'])

# Information for each "grade" including, potential, costs, historical extraction:
potentials = {'dross': [50000*30, 0, 0],
              'post-consumer': [1000000*30, 0, 0]}

# in case you need to add the new unit to the platform
mp.add_unit('USD/t')

# Adding required information via MESSAGEix sets
for grade in potentials.keys():

    # Adding resource potentials
    # The index sets for resource_volume are ['node', 'commodity', 'grade', 'value', 'unit']
    df = pd.DataFrame({'node': [country],
                       'commodity': commodity,
                       'grade': grade,
                       'value': potentials[grade][0],
                       'unit': 't'})
    scenario.add_par('resource_volume', df)
    
    # Adding resource costs
    # The index sets for resource_cost are ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': model_horizon,
                       'value': potentials[grade][1],
                       'unit': 'USD/t'})
    scenario.add_par('resource_cost', df)
    
    # Adding historical extraction
    # The index sets for historical_extraction are ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': [2020],
                       'value':  potentials[grade][2],
                       'unit': 't'})
    scenario.add_par('historical_extraction', df)
    

df = make_df(base_input, technology='alumino', commodity='scrap', 
                   level='recovery', value=1, unit='%')

scenario.add_par('input', df)

In [19]:
from message_ix import log

log.info('version number prior to commit: {}'.format(scenario.version))

scenario.commit(comment='basic model of project')

log.info('version number prior committing to the database: {}'.format(scenario.version))

In [20]:
scenario.set_as_default()
scenario.solve()
scenario.var('OBJ')['lvl']

#total cost of the cheapest technology (considering emission restrictions) up to and including the year 2050.

29430286336.0

Plotting results:

In [21]:
# Create a Reporter object to describe and carry out reporting
# calculations and operations (like plotting) based on `scenario`
from message_ix.reporting import Reporter

rep = Reporter.from_scenario(scenario)

# Add keys like "plot activity" to describe reporting operations.
# See tutorial/utils/plotting.py
from message_ix.util.tutorial import prepare_plots

prepare_plots(rep)

In [22]:
rep.get("ACT")

nl      t           yv    ya    m         h   
Norway  residues    2030  2030  standard  year         0.000000
                    2040  2040  standard  year         0.000000
                    2050  2050  standard  year         0.000000
        SAF         2030  2030  standard  year     52327.527144
                          2040  standard  year         0.000000
                          2050  standard  year         0.000000
                    2040  2040  standard  year         0.000000
                          2050  standard  year         0.000000
                    2050  2050  standard  year         0.000000
        alumino     2030  2030  standard  year    404296.828970
                          2040  standard  year         0.000000
                          2050  standard  year         0.000000
                    2040  2040  standard  year    636732.436309
                          2050  standard  year         0.000000
                    2050  2050  standard  year    898173.

In [23]:
rep.get("EMISS")

n       e    type_tec  y   
World   CO2  all       2030    2.391577e+06
                       2040    3.247335e+06
                       2050    4.580687e+06
Norway  CO2  all       2030    2.391577e+06
                       2040    3.247335e+06
                       2050    4.580687e+06
Name: EMISS, dtype: float64

In [24]:
mp.close_db()